In [3]:
import pandas as pd

In [4]:
qpd_df = pd.read_csv("data/qinn_processed_data.csv")

df = qpd_df.copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   room_num              6665 non-null   int64  
 1   room_type             6665 non-null   object 
 2   customer_id           6665 non-null   object 
 3   stay_duration         6663 non-null   object 
 4   total_price           6665 non-null   float64
 5   price                 6665 non-null   float64
 6   tax                   6665 non-null   float64
 7   customer_name         6665 non-null   object 
 8   checkin_datetime      6665 non-null   object 
 9   checkout_datetime     6665 non-null   object 
 10  days                  6665 non-null   float64
 11  hours                 6665 non-null   float64
 12  checkin_month         6665 non-null   int64  
 13  checkin_day           6665 non-null   int64  
 14  checkin_year          6665 non-null   int64  
 15  checkin_time         